<a href="https://colab.research.google.com/github/Ahndoyun/Project_Game_Analysis/blob/main/N422a_Distributed_Representation_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="right" src="https://ds-cs-images.s3.ap-northeast-2.amazonaws.com/Codestates_Fulllogo_Color.png" width=100>

## *AIB / SECTION 4 / SPRINT 2 / NOTE 2*

# 📝 Assignment

---


# Distributed_Representation

## Code

캐글의 [SMS Spam dataset](https://www.kaggle.com/uciml/sms-spam-collection-dataset) 에 사전 학습된 Word2Vec 임베딩 벡터를 적용하여 분류해봅시다.<br/>
세션 노트에 있었던 단어 임베딩 벡터를 평균내어 분류하는 방법을 적용해봅시다.

In [4]:
!pip install gensim --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import gensim

gensim.__version__

'4.2.0'

In [6]:
import gensim.downloader as api

wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


### 1. Word2Vec과 코사인 유사도

word2vec을 이용해 구한 'data'와 'science'임베딩 값의 코사인 유사도를 구해보겠습니다. sklearn의 cosine_similarity를 이용하겠습니다.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity(wv['data'].reshape(1,-1), wv['science'].reshape(1,-1))

array([[0.1575913]], dtype=float32)

In [9]:
round(cosine_similarity(wv['data', 'science'])[0][1],3)

0.158

문항 1) 위에서 구한 코사인 유사도를 소수점 3째자리까지 입력하세요

In [10]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer

### 2. 텍스트 분류

### 1) 데이터 전처리
    
- 데이터셋을 데이터프레임으로 읽어옵니다 `encoding = 'latin-1'` 을 사용합니다.
- 필요없는 열(column)을 삭제합니다.
- LabelEncoder를 사용하여 label 전처리를 해줍니다. 

In [12]:
from google.colab import files

file = files.upload()

Saving spam.csv to spam.csv


In [13]:
df = pd.read_csv('spam.csv', encoding = 'latin-1')

In [14]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [15]:
df = df[['v1', 'v2']]
df

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [16]:
df['v1'].value_counts()

ham     4825
spam     747
Name: v1, dtype: int64

In [17]:
encoder = LabelEncoder()
df['v1'] = encoder.fit_transform(df['v1'])
df

,v1,v2
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,1,This is the 2nd time we have tried 2 contact u...
5568,0,Will Ì_ b going to esplanade fr home?
5569,0,"Pity, * was in mood for that. So...any other s..."
5570,0,The guy did some bitching but I acted like i'd...


### 2) 텍스트 분류를 수행해주세요.

- 데이터셋 split시 test_size의 비율은 15%로, `random_state = 42` 로 설정합니다. 
- Tokenizer의 `num_words = 1000` 으로 설정합니다.
- pad_sequence의 `maxlen=150` 으로 설정합니다.
- 학습 시, 파라미터는 `batch_size=64, epochs=10, validation_split=0.2` 로 설정합니다.
- evaluate 했을 때의 loss와 accuarcy를 [loss, acc] 형태로 입력해주세요. Ex) [0.4321, 0.8765]

In [18]:
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df['v2'], df['v1'], test_size = 0.15, random_state = 42)
X_train.shape, X_test.shape

((4736,), (836,))

In [20]:
tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(X_train)

vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8210


In [21]:
X_train_encoded = tokenizer.texts_to_sequences(X_train)
X_test_encoded = tokenizer.texts_to_sequences(X_test)

max_len = max(len(sent) for sent in X_test_encoded)
print(max_len)
print(f'학습 데이터에 있는 문서의 평균 토큰 수: {np.mean([len(sent) for sent in X_train], dtype=int)}')

171
학습 데이터에 있는 문서의 평균 토큰 수: 79


In [22]:
#padding
maxlen = 150
X_train = pad_sequences(X_train_encoded, maxlen=maxlen, padding='post')
X_test = pad_sequences(X_test_encoded, maxlen=maxlen, padding='post')

y_train = np.array(y_train)
y_test = np.array(y_test)

In [23]:
embedding_matrix = np.zeros((vocab_size, 300))

print(np.shape(embedding_matrix))

(8210, 300)


In [24]:
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp

In [25]:
#신경망모델
model = Sequential([
    Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=maxlen, trainable=False),
    GlobalAveragePooling1D(),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

Epoch 1/10
60/60 [==============================] - 2s 16ms/step - loss: 0.6707 - acc: 0.8643 - val_loss: 0.6509 - val_acc: 0.8660
Epoch 2/10
60/60 [==============================] - 0s 8ms/step - loss: 0.6352 - acc: 0.8648 - val_loss: 0.6175 - val_acc: 0.8660
Epoch 3/10
60/60 [==============================] - 0s 7ms/step - loss: 0.6043 - acc: 0.8648 - val_loss: 0.5886 - val_acc: 0.8660
Epoch 4/10
60/60 [==============================] - 0s 7ms/step - loss: 0.5778 - acc: 0.8648 - val_loss: 0.5633 - val_acc: 0.8660
Epoch 5/10
60/60 [==============================] - 0s 7ms/step - loss: 0.5550 - acc: 0.8648 - val_loss: 0.5419 - val_acc: 0.8660
Epoch 6/10
60/60 [==============================] - 0s 7ms/step - loss: 0.5353 - acc: 0.8648 - val_loss: 0.5234 - val_acc: 0.8660
Epoch 7/10
60/60 [==============================] - 0s 7ms/step - loss: 0.5182 - acc: 0.8648 - val_loss: 0.5071 - val_acc: 0.8660
Epoch 8/10
60/60 [==============================] - 0s 7ms/step - loss: 0.5035 - acc: 0.8

In [26]:
model.evaluate(X_test, y_test)

27/27 [==============================] - 0s 5ms/step - loss: 0.4697 - acc: 0.8708


[0.4696813225746155, 0.8708133697509766]

### 3)Word2Vec에서의 OOV 문제

```
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
```
Lecture Note에 있는 위의 코드를 변형하여, OOV 개수를 확인해주세요.
- tokenizer는 위에서 활용한 tokenizer를 그대로 사용하겠습니다.
- Tip : dictionary를 활용하거나, Counter를 활용해보세요.

In [27]:
def get_vector(word):
    """
    해당 word가 word2vec에 있는 단어일 경우 임베딩 벡터를 반환
    """
    if word in wv:
        return wv[word]
    else:
        return None
 
oov = []
for word, i in tokenizer.word_index.items():
    temp = get_vector(word)
    if temp is not None:
        embedding_matrix[i] = temp
    else:
      oov.append(word)

print(len(oov))


2419
